In [1]:
%matplotlib inline
import numpy as np
from pymt.models import PRMSSurface, PRMSSoil, PRMSGroundwater, PRMSStreamflow
from pathlib import Path
import geopandas as gpd
import pandas as pd
import helper

➡ models: Avulsion, Plume, Sedflux3D, Subside, Rafem, PRMSSurface, PRMSStreamflow, PRMSSoil, PRMSGroundwater, FrostNumber, Ku, Hydrotrend, GIPL, ECSimpleSnow, Cem, Waves


In [2]:
run_dir = '../prms/pipestem'
config_surf= 'control_surface.simple1'
config_soil = 'control_soil.simple1'

print(Path(run_dir).exists())
print((Path(run_dir) / config_surf).exists())
print((Path(run_dir) / config_soil).exists())

msurf = PRMSSurface()
msoil = PRMSSoil()

print(msurf.name, msoil.name)


True
True
True
prms6-surface-BMI prms6-BMI-SOIL


In [3]:
msurf.initialize(config_surf, run_dir)
msoil.initialize(config_soil, run_dir)

In [4]:
soil_input_cond_vars = ['soil_rechr_chg', 'soil_moist_chg']

surf2soil_vars = ['hru_ppt', 'dprst_evap_hru', 
                   'dprst_seep_hru', 'infil', 'sroff', 'potet', 'hru_intcpevap', 
                   'snow_evap', 'snowcov_area', 'soil_rechr', 'soil_rechr_max', 
                   'soil_moist', 'soil_moist_max', 'hru_impervevap']

soilinput_params = ['hru_area_perv', 'hru_frac_perv', 'srunoff_updated_soil', 'transp_on']


soil2surf_vars = ['infil', 'sroff', 'soil_rechr', 'soil_moist']

# infil, sroff, soil_moist, soil_rechr are calculated in both surface and soil
# ptr_infil = msurf.get_value_ptr('infil')
hru_ppt_ptr = msurf.get_value_ptr('hru_ppt')
dprst_evap_hru_ptr = msurf.get_value_ptr('dprst_evap_hru')
dprst_seep_hru_ptr = msurf.get_value_ptr('dprst_seep_hru')
infil_ptr = msurf.get_value_ptr('infil')
sroff_ptr = msurf.get_value_ptr('sroff')
potet_ptr = msurf.get_value_ptr('potet')
hru_intcpevap_ptr = msurf.get_value_ptr('hru_intcpevap')
snow_evap_ptr = msurf.get_value_ptr('snow_evap')
snowcov_area_ptr = msurf.get_value_ptr('snowcov_area')
soil_rechr_ptr = msurf.get_value_ptr('soil_rechr')
soil_rechr_max_ptr = msurf.get_value_ptr('soil_rechr_max')
soil_moist_ptr = msurf.get_value_ptr('soil_moist')
soil_moist_max_ptr = msurf.get_value_ptr('soil_moist_max')
hru_impervevap_ptr = msurf.get_value_ptr('hru_impervevap')
transp_on_ptr = msurf.get_value_ptr('transp_on')

soil_rechr_chg_ptr = msurf.get_value_ptr('soil_rechr_chg')
soil_moist_chg_ptr = msurf.get_value_ptr('soil_moist_chg')

soil_infil_ptr = msoil.get_value_ptr('infil')
soil_sroff_ptr = msoil.get_value_ptr('sroff')
soil_soil_moist_ptr = msoil.get_value_ptr('soil_moist')
soil_soil_rechr_ptr = msoil.get_value_ptr('soil_rechr')

def updateSoilPtr(msoil):
    msoil.set_value('hru_ppt', hru_ppt_ptr)
    msoil.set_value('dprst_evap_hru', dprst_evap_hru_ptr)
    msoil.set_value('dprst_seep_hru', dprst_seep_hru_ptr)
    msoil.set_value('infil', infil_ptr)
    msoil.set_value('sroff', sroff_ptr)
    msoil.set_value('potet', potet_ptr)
    msoil.set_value('hru_intcpevap', hru_intcpevap_ptr)
    msoil.set_value('snow_evap', snow_evap_ptr)
    msoil.set_value('snowcov_area', snowcov_area_ptr)
    msoil.set_value('soil_rechr', soil_rechr_ptr)
    msoil.set_value('soil_rechr_max', soil_rechr_max_ptr)
    msoil.set_value('soil_moist', soil_moist_ptr)
    msoil.set_value('soil_moist_max', soil_moist_max_ptr)
    msoil.set_value('hru_impervevap', hru_impervevap_ptr)
    
    msoil.set_value('transp_on', transp_on_ptr)
    msoil.set_value('srunoff_updated_soil', msurf.get_value('srunoff_updated_soil'))
    
#     msoil.set_value('soil_moist_chg', msurf.get_value('soil_moist_chg'))
#     msoil.set_value('soil_rechr_chg', msurf.get_value('soil_rechr_chg'))
    
def updateSurfPtr(msurf):
    msurf.set_value('infil', soil_infil_ptr)
    msurf.set_value('sroff', soil_sroff_ptr)
    msurf.set_value('soil_moist', soil_soil_moist_ptr)
    msurf.set_value('soil_rechr', soil_soil_rechr_ptr)

def soil2surface(msoil, msurf, exch_vars):
    for var in exch_vars:
        msurf.set_value(var, msoil.get_value(var))


def soilinput(msurf, msoil, exch_vars):
    for var in exch_vars:
        print(var)
        msoil.set_value(var, msurf.get_value(var))
            
def soilexchange(msurf, msoil, exch_vars, cond_vars, dprst_flag, imperv_flag):
    for var in exch_vars:
        print(var)
        msoil.set_value(var, msurf.get_value_ptr(var))
    if dprst_flag in [1, 3] or imperv_flag in [1, 3]:
        for var in cond_vars:
            msoil.set_value(var, msurf.get_value_ptr(var))

# def soil2surface(msoil, msurf, exch_vars):
#     for var in exch_vars:
#         msurf.set_value(var, msoil.get_value(var))
        
# def gwinput(msurf, msoil, mgw, surf_vars, soil_vars):
#     for var in surf_vars:
#         mgw.set_value(var, msurf.get_value(var))
#     for var in soil_vars:
#         mgw.set_value(var, msoil.get_value(var))
        
# def sfinput(msurf, msoil, mgw, msf, surf_vars, soil_vars, gw_vars):
#     for var in surf_vars:
#         msf.set_value(var, msurf.get_value(var))
#     for var in soil_vars:
#         msf.set_value(var, msoil.get_value(var))
#     for var in gw_vars:
#         msf.set_value(var, mgw.get_value(var))

dprst_flag = msoil.get_value('dyn_dprst_flag')
imperv_flag = msoil.get_value('dyn_imperv_flag')
print(f'dprst_flag/imperv_flag: {dprst_flag}/{imperv_flag}')
soilinput(msurf, msoil, soilinput_params)
# soilexchange(msurf, msoil, surf2soil_vars, soil_input_cond_vars, dprst_flag, imperv_flag)
def update_coupled(msurf, msoil):
    msurf.update()
    updateSoilPtr(msoil)
    msoil.update()
    updateSurfPtr(msurf)
#     soil2surface(msurf, msoil, soil2surf_vars)
          

dprst_flag/imperv_flag: [0]/[0]
hru_area_perv
hru_frac_perv
srunoff_updated_soil
transp_on


In [5]:
ptr_infil = msurf.get_value_ptr('infil')
for i in range(int(msurf.start_time),int(msurf.end_time)):
    update_coupled(msurf, msoil)
    tmp = msurf.get_value('infil')[13]
    tmp2 = msoil.get_value('infil')[13]
    print(f'surface/soil infil value/surface_ptr: {tmp}/{tmp2}/{ptr_infil[13]}')


surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0


surface/soil infil value/surface_ptr: 0.6263583302497864/0.6263583302497864/0.6263583302497864
surface/soil infil value/surface_ptr: 0.9505910873413086/0.9505910873413086/0.9505910873413086
surface/soil infil value/surface_ptr: 0.10484541952610016/0.10484541952610016/0.10484541952610016
surface/soil infil value/surface_ptr: 0.32599884271621704/0.32599884271621704/0.32599884271621704
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.13910534977912903/0.13910534977912903/0.13910534977912903
surface/soil infil value/surface_ptr: 0.39949512481689453/0.39949512481689453/0.39949512481689453
surface/soil infil value/surface_ptr: 0.2653985917568207/0.2653985917568207/0.2653985917568207
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil valu

surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.9450445175170898/0.9450445175170898/0.9450445175170898
surface/soil infil value/surface_ptr: 0.7464957237243652/0.7464957237243652/0.7464957237243652
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/so

surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.03641219809651375/0.03641219809651375/0.03641219809651375
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
su

surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.05512408912181854/0.05512408912181854/0.05512408912181854
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.26811760663986206/0.26811760663986206/0.26811760663986206
surface/soil infil value/surface_ptr: 0.2832677364349365/0.2832677364349365/0.2832677364349365
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.24269668757915497/0.24269668757915497/0.24269668757915497
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soi

surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0
surface/soil infil value/surface_ptr: 0.0/0.0/0.0


In [ ]:
# CSDMS JupyterHub set path to HRU and streamsegment shapefiles
hru_shp = '/data/prms/GIS/nhru_10U.shp'
hru_strmseg = '/data/prms/GIS/nsegment_10U.shp'
# set path to Gridmet weights file for mapping Gridmet gridded data to HRU
weight_file = '/data/prms/weights.csv'

# If using notebook not in CSDMS JupyterHub.  See README for instruction on where to 
# get the data and  uncomment out the following lines
# hru_shp = './GIS/nhru_10U.shp'
# hru_strmseg = './GIS/nsegment_10U.shp'
# # set path to Gridmet weights file for mapping Gridmet gridded data to HRU
# weight_file = './GIS/weights.csv'

gdf_ps = helper.get_gdf(hru_shp, msurf)

In [ ]:
msurf.finalize()
msoil.finalize()

In [ ]:
import xarray as xr
soil_file = Path('../prms/pipestem/output/summary_soil_daily.nc')
surf_file = Path('../prms/pipestem/output/summary_surf_daily.nc')

prms_file = Path('../prms/pipestem/output/summary_prms_daily.nc')
clim_file = Path('../prms/pipestem/daymet.nc')
dsoil = xr.open_dataset(soil_file)
dprms = xr.open_dataset(prms_file)
dsurf = xr.open_dataset(surf_file)

clim = xr.open_dataset(clim_file)

In [ ]:
sim_start_date = dsoil.time.min()
sim_end_date = dsoil.time.max()

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
from helper import bmi_prms6_value_plot, bmi_prms6_residual_plot
import matplotlib.pyplot as plt
import matplotlib
t_hru = 13

fig, ax = plt.subplots(ncols=2)
bmi_prms6_value_plot(dsoil, t_hru, 'infil', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_value_plot(dprms, t_hru, 'infil', 'prms', sim_start_date, sim_end_date, ax[0])
bmi_prms6_residual_plot(dsoil, dprms, t_hru, 'infil', 'prms', sim_start_date, sim_end_date, ax[1])
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(ncols=2)
bmi_prms6_value_plot(dsoil, t_hru, 'sroff', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_value_plot(dprms, t_hru, 'sroff', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_residual_plot(dsoil, dprms, t_hru, 'sroff', 'prms', sim_start_date, sim_end_date, ax[1])
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(ncols=2)
bmi_prms6_value_plot(dsoil, t_hru, 'soil_moist', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_value_plot(dprms, t_hru, 'soil_moist', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_residual_plot(dsoil, dprms, t_hru, 'soil_moist', 'prms', sim_start_date, sim_end_date, ax[1])
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(ncols=2)
bmi_prms6_value_plot(dsoil, t_hru, 'soil_rechr', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_value_plot(dprms, t_hru, 'soil_rechr', 'soil-bmi', sim_start_date, sim_end_date, ax[0])
bmi_prms6_residual_plot(dsoil, dprms, t_hru, 'soil_rechr', 'prms', sim_start_date, sim_end_date, ax[1])
plt.gca().ticklabel_format(axis='y', style='plain', useOffset=False)
plt.tight_layout()
plt.show()
